In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

1、获取股票列表

In [2]:
df = pd.read_excel('./股票列表.xlsx', engine='openpyxl')
code_list = []
# 遍历第二列中的股票代码
for stock_code in df['代码']:
    old_code = stock_code
    if stock_code.startswith('SH'):
        stock_code = f"{stock_code[2:]}.{'SS'}"
    elif stock_code.startswith('SZ'):
        stock_code = f"{stock_code[2:]}.{'SZ'}"
    code_list.append(stock_code)
code_list

['300905.SZ',
 '300561.SZ',
 '300287.SZ',
 '300125.SZ',
 '300086.SZ',
 '300386.SZ',
 '000566.SZ',
 '000850.SZ',
 '002036.SZ',
 '603123.SS',
 '600676.SS',
 '600841.SS',
 '601162.SS',
 '600148.SS',
 '600650.SS',
 '600635.SS',
 '605218.SS',
 '600847.SS',
 '000062.SZ',
 '603883.SS',
 '002104.SZ',
 '001298.SZ',
 '000750.SZ',
 '000678.SZ',
 '000627.SZ',
 '603306.SS',
 '600838.SS',
 '600611.SS',
 '603626.SS',
 '002823.SZ',
 '600826.SS',
 '002640.SZ',
 '002520.SZ',
 '600828.SS',
 '002717.SZ',
 '300736.SZ',
 '301607.SZ',
 '002456.SZ',
 '300641.SZ',
 '600418.SS',
 '300209.SZ',
 '300758.SZ',
 '605289.SS',
 '001696.SZ',
 '300077.SZ',
 '300464.SZ',
 '600686.SS',
 '002388.SZ',
 '601890.SS',
 '601086.SS',
 '300013.SZ',
 '605598.SS',
 '002827.SZ',
 '600769.SS',
 '002208.SZ',
 '600576.SS',
 '300518.SZ',
 '300619.SZ',
 '300436.SZ',
 '603036.SS',
 '300068.SZ',
 '002312.SZ',
 '002528.SZ',
 '600898.SS',
 '000536.SZ',
 '000712.SZ',
 '600358.SS',
 '000908.SZ',
 '000584.SZ',
 '603559.SS',
 '605228.SS',
 '6008

2、定义起始日期

In [3]:
start_date = '2015-01-01'
end_date = '2024-08-31'

3、获取所有股票的数据整合到一个文件中

In [4]:
df = pd.read_csv('./all_data.csv')
df

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,code
0,0,2020-11-05,55.000000,62.150002,51.000000,51.029999,47.681602,21472552.0,300905.SZ
1,1,2020-11-06,47.005001,51.520000,46.505001,48.599998,45.411049,13282442.0,300905.SZ
2,2,2020-11-09,48.500000,49.845001,47.119999,48.740002,45.541870,11263252.0,300905.SZ
3,3,2020-11-10,48.000000,49.439999,47.000000,47.250000,44.149632,8399338.0,300905.SZ
4,4,2020-11-11,46.900002,49.055000,46.595001,47.700001,44.570107,8407878.0,300905.SZ
...,...,...,...,...,...,...,...,...,...
195256,195256,2024-08-26,10.470000,11.520000,10.420000,11.520000,11.520000,10574100.0,600506.SS
195257,195257,2024-08-27,11.500000,11.770000,11.220000,11.280000,11.280000,13423101.0,600506.SS
195258,195258,2024-08-28,11.380000,12.300000,11.210000,11.600000,11.600000,13100100.0,600506.SS
195259,195259,2024-08-29,11.530000,11.680000,11.160000,11.350000,11.350000,9205800.0,600506.SS


4、数据清洗

In [5]:
# 过滤掉code列中以'300'开头的行
df = df[~df['code'].str.startswith('300')]
df = df[~df['code'].str.startswith('688')]
df = df[~df['code'].str.startswith('8')]

df

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,code
11834,11834,2015-01-05,7.440,7.490,7.290,7.385,7.192119,12799906.0,000566.SZ
11835,11835,2015-01-06,7.390,7.710,7.365,7.595,7.396634,14522622.0,000566.SZ
11836,11836,2015-01-07,7.645,8.050,7.645,7.755,7.552455,24060822.0,000566.SZ
11837,11837,2015-01-08,7.800,7.885,7.605,7.705,7.503761,10945550.0,000566.SZ
11838,11838,2015-01-09,7.845,8.090,7.845,7.945,7.737493,25795612.0,000566.SZ
...,...,...,...,...,...,...,...,...,...
195256,195256,2024-08-26,10.470,11.520,10.420,11.520,11.520000,10574100.0,600506.SS
195257,195257,2024-08-27,11.500,11.770,11.220,11.280,11.280000,13423101.0,600506.SS
195258,195258,2024-08-28,11.380,12.300,11.210,11.600,11.600000,13100100.0,600506.SS
195259,195259,2024-08-29,11.530,11.680,11.160,11.350,11.350000,9205800.0,600506.SS


In [6]:
df.set_index('code', inplace=True)

In [7]:
# 计算每只股票的当日涨幅
df['Pct_Chg'] = df.groupby(level=0)['Close'].pct_change()
# 计算3天涨幅
df['Pct_Chg_3d'] = df.groupby(level=0)['Close'].pct_change(periods=3) * 100
# 计算5天涨幅
df['Pct_Chg_5d'] = df.groupby(level=0)['Close'].pct_change(periods=5) * 100
# 计算10天涨幅
df['Pct_Chg_10d'] = df.groupby(level=0)['Close'].pct_change(periods=10) * 100
# 计算20天涨幅
df['Pct_Chg_20d'] = df.groupby(level=0)['Close'].pct_change(periods=20) * 100
# 将涨幅数据转换为百分比形式
df['Pct_Chg'] *= 100
# 去除Pct_Chg为空的值
df = df.dropna(subset=['Pct_Chg','Pct_Chg_3d', 'Pct_Chg_5d', 'Pct_Chg_10d', 'Pct_Chg_20d'])
df

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Pct_Chg,Pct_Chg_3d,Pct_Chg_5d,Pct_Chg_10d,Pct_Chg_20d
code,,,,,,,,,,,,,
000566.SZ,11854,2015-02-02,9.060,9.385,9.055,9.220,8.979192,7526534.0,0.217396,-2.793879,0.875273,17.452234,24.847664
000566.SZ,11855,2015-02-03,9.250,9.710,9.220,9.500,9.251880,14995184.0,3.036873,-0.575618,-1.144637,11.567817,25.082294
000566.SZ,11856,2015-02-04,9.545,9.700,9.500,9.550,9.300573,12974634.0,0.526318,3.804352,0.685298,6.644336,23.146358
000566.SZ,11857,2015-02-05,9.610,9.700,9.400,9.460,9.212924,14038160.0,-0.942410,2.603034,-0.994247,5.875765,22.777419
000566.SZ,11858,2015-02-06,9.375,9.375,8.925,9.075,8.837979,14307994.0,-4.069770,-4.473686,-1.358696,3.655051,14.222777
...,...,...,...,...,...,...,...,...,...,...,...,...,...
600506.SS,195256,2024-08-26,10.470,11.520,10.420,11.520,11.520000,10574100.0,10.028655,7.764274,6.864571,0.787403,1.497798
600506.SS,195257,2024-08-27,11.500,11.770,11.220,11.280,11.280000,13423101.0,-2.083340,8.045979,6.214688,-1.398600,-1.484717
600506.SS,195258,2024-08-28,11.380,12.300,11.210,11.600,11.600000,13100100.0,2.836885,10.792742,8.512636,4.598740,-0.684931


5、和上证指数数据合并

In [8]:
df_szzs = pd.read_csv('000001.csv')
df_szzs.set_index('code', inplace=True)
# 计算每只股票的当日涨幅
df_szzs['Pct_Chg_szzs'] = df_szzs.groupby(level=0)['Close'].pct_change()
# 将涨幅数据转换为百分比形式
df_szzs['Pct_Chg_szzs'] *= 100
# 去除Pct_Chg为空的值
df_szzs = df_szzs.dropna(subset=['Pct_Chg_szzs'])

df_szzs

,Date,Open,High,Low,Close,Adj Close,Volume,Pct_Chg_szzs
code,,,,,,,,
000001.SS,2015-01-06,3330.799072,3394.224121,3303.184082,3351.446045,3351.446045,501700,0.027667
000001.SS,2015-01-07,3326.648926,3374.895996,3312.210938,3373.954102,3373.954102,391900,0.671592
000001.SS,2015-01-08,3371.957031,3381.565918,3285.094971,3293.456055,3293.456055,371100,-2.385867
000001.SS,2015-01-09,3276.965088,3404.833984,3267.509033,3285.412109,3285.412109,410200,-0.244240
000001.SS,2015-01-12,3258.212891,3275.185059,3191.582031,3229.315918,3229.315918,322100,-1.707432
...,...,...,...,...,...,...,...,...
000001.SS,2024-08-26,2855.474121,2860.655029,2844.383057,2855.520020,2855.520020,227400,0.040363
000001.SS,2024-08-27,2850.876953,2853.780029,2843.157959,2848.730957,2848.730957,230700,-0.237752
000001.SS,2024-08-28,2844.326904,2852.562012,2830.961914,2837.428955,2837.428955,228700,-0.396738


In [9]:
# 重置索引，以便合并
df = df.reset_index()
df_szzs = df_szzs.reset_index()
# 合并两个DataFrame
# 这里我们按照'Date'列进行合并，并且使用左连接，这样只会保留df中的日期
df = df.merge(df_szzs[['Date', 'Pct_Chg_szzs']], on='Date', how='left')
df

,code,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Pct_Chg,Pct_Chg_3d,Pct_Chg_5d,Pct_Chg_10d,Pct_Chg_20d,Pct_Chg_szzs
0,000566.SZ,11854,2015-02-02,9.060,9.385,9.055,9.220,8.979192,7526534.0,0.217396,-2.793879,0.875273,17.452234,24.847664,-2.556190
1,000566.SZ,11855,2015-02-03,9.250,9.710,9.220,9.500,9.251880,14995184.0,3.036873,-0.575618,-1.144637,11.567817,25.082294,2.448836
2,000566.SZ,11856,2015-02-04,9.545,9.700,9.500,9.550,9.300573,12974634.0,0.526318,3.804352,0.685298,6.644336,23.146358,-0.960434
3,000566.SZ,11857,2015-02-05,9.610,9.700,9.400,9.460,9.212924,14038160.0,-0.942410,2.603034,-0.994247,5.875765,22.777419,-1.184420
4,000566.SZ,11858,2015-02-06,9.375,9.375,8.925,9.075,8.837979,14307994.0,-4.069770,-4.473686,-1.358696,3.655051,14.222777,-1.932837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157393,600506.SS,195256,2024-08-26,10.470,11.520,10.420,11.520,11.520000,10574100.0,10.028655,7.764274,6.864571,0.787403,1.497798,0.040363
157394,600506.SS,195257,2024-08-27,11.500,11.770,11.220,11.280,11.280000,13423101.0,-2.083340,8.045979,6.214688,-1.398600,-1.484717,-0.237752
157395,600506.SS,195258,2024-08-28,11.380,12.300,11.210,11.600,11.600000,13100100.0,2.836885,10.792742,8.512636,4.598740,-0.684931,-0.396738
157396,600506.SS,195259,2024-08-29,11.530,11.680,11.160,11.350,11.350000,9205800.0,-2.155172,-1.475695,8.716483,1.520573,-3.321971,-0.504788


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

线性回归预测当日涨幅和大盘涨幅对3日收益率的影响          
探究大盘下跌时股价逆势上涨是否更容易导致后续的股价增长

In [11]:
def linear_model_1_Normal_Equation(df,x_target):
     """
     线性回归：正规方程
     """
     # 1. 选择特征变量和目标变量
     target = x_target  # 目标变量
     features = ['Pct_Chg','Pct_Chg_szzs']  # 特征变量

     # 2. 数据集划分
     x_train, x_test, y_train, y_test = train_test_split(df[features], df[target]-df['Pct_Chg'], random_state=2, test_size=0.2)

     # 3. 特征工程：标准化
     transformer = StandardScaler()
     
     # fit()函数用于计算训练数据的均值和标准差，以便对训练数据进行标准化
     x_train = transformer.fit_transform(x_train)

     # transform()函数用于使用相同的均值和标准差来对测试数据进行标准化。这确保了我们使用相同的标准化方法来处理训练和测试数据。
     x_test = transformer.transform(x_test)
     
     # 4. 机器学习：线性回归（正规方程）
     estimator = LinearRegression()
     estimator.fit(x_train, y_train)
     
     # 5. 模型评估
     ## 5.1 获取系数
     y_predict = estimator.predict(x_test)
     print(f"预测值为：{y_predict}")
     print(f"模型中的系数为：{estimator.coef_}")
     print(f"模型中的偏置为：{estimator.intercept_}")
     
     ## 5.2 评价：均方误差
     error_mse = mean_squared_error(y_test, y_predict)
     print(f"均方误差为：{error_mse}")
     
     return None


In [12]:
x_target = 'Pct_Chg_3d'
linear_model_1_Normal_Equation(df,x_target)

预测值为：[-0.0165636   0.06115216 -0.1339683  ...  0.11357258 -0.08698684
 -0.78013654]
模型中的系数为：[ 0.46891662 -0.26620151]
模型中的偏置为：0.03426338312478401
均方误差为：21.15462901572009


探究大盘下跌情况下逆势涨停的股票后续是否可能会更大幅度上涨

In [13]:
df_best = df[df['Pct_Chg'] > 9.8]
x_target = 'Pct_Chg_3d'
print(f'************逆势上涨直至涨停的股票***************')
linear_model_1_Normal_Equation(df_best,x_target)
df_lost = df[df['Pct_Chg'] < 9.8]
print(f'************逆势上涨但未涨停的股票***************')
linear_model_1_Normal_Equation(df_lost,x_target)

************逆势上涨直至涨停的股票***************
预测值为：[ 6.83981283e+00 -6.08153469e-01  3.81918723e+00  4.76366493e+00
  5.00472826e+00  1.68610724e+00  4.64273873e+00 -1.87277553e+00
  3.66329572e+00 -1.42333087e-01  4.81178153e+00  1.85001830e+00
  3.28355062e-01  5.06765972e+00  3.90222062e+00 -2.68377753e-02
  5.42658857e+00  1.24401274e+00  3.82386079e+00 -2.44778947e-01
 -2.24618349e+00  4.11231977e+00  2.22352967e+00  4.93250851e+00
  3.21335515e+00  4.86836722e+00  4.06200536e+00  4.62972917e+00
  4.64005546e+00  1.84932027e+00  3.82140620e+00  4.66331466e+00
  5.95613523e+00  4.02691125e+00  6.11446998e+00  3.94136389e+00
 -1.60788685e-01  3.61708833e+00  6.82992126e+00  5.81425014e+00
  4.94316263e+00  4.41428246e+00  5.87073688e+00  1.52258003e+00
  1.38819250e+00  4.75991164e+00  4.31511577e+00  3.90360848e+00
  5.26817176e+00  4.08244314e+00  4.12261324e+00 -4.20286933e+00
  4.02330553e+00  2.09021999e+00  6.07519454e+00  2.15660449e+00
  3.84792743e+00  5.07195293e+00  5.30593168e+

采用逻辑回归算法进行预测

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [15]:
def logistic_regression_model(data, feature_cols, target_col):
    """
    训练逻辑回归模型并评估性能

    参数:
    data (DataFrame): 包含特征和目标变量的数据集
    feature_cols (list): 特征列名称的列表
    target_col (str): 目标列名称

    返回:
    dict: 包含模型评估指标的字典
    """
    # 提取特征和目标变量
    X = data[feature_cols]
    y = data[target_col]

    # 分割数据集为训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 创建逻辑回归模型
    log_regressor = LogisticRegression()
    log_regressor.fit(X_train, y_train)

    # 进行预测
    y_pred = log_regressor.predict(X_test)

    # 计算准确率
    accuracy = accuracy_score(y_test, y_pred)
    
    # 混淆矩阵
    conf_matrix = confusion_matrix(y_test, y_pred)

    # 分类报告
    class_report = classification_report(y_test, y_pred)

    # 返回结果
    return {
        'accuracy': accuracy,
        'confusion_matrix': conf_matrix,
        'classification_report': class_report
    }

探究大盘和当日股价变化对后续股价的预测准确率

In [20]:
print("***********3日*************")
df['Target_3d'] = (df['Pct_Chg_3d']-df['Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_3d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("***********5日*************")
df['Target_5d'] = (df['Pct_Chg_5d']-df['Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_5d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("***********10日*************")
df['Target_10d'] = (df['Pct_Chg_10d']-df['Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_10d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("**********20日*************")
df['Target_20d'] = (df['Pct_Chg_20d']-df['Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_20d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("**********20内的最高价日*************")
columns_to_check = ['Pct_Chg_3d', 'Pct_Chg_5d', 'Pct_Chg_10d', 'Pct_Chg_20d']
df['Max_Pct_Chg'] = df[columns_to_check].max(axis=1)
df['Target_max'] = (df['Max_Pct_Chg']-df['Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_max')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])

***********3日*************
模型准确率: 0.5384053367217281
混淆矩阵:
[[15921   788]
 [13743  1028]]
分类报告:
              precision    recall  f1-score   support

           0       0.54      0.95      0.69     16709
           1       0.57      0.07      0.12     14771

    accuracy                           0.54     31480
   macro avg       0.55      0.51      0.41     31480
weighted avg       0.55      0.54      0.42     31480

***********5日*************
模型准确率: 0.535546378653113
混淆矩阵:
[[15898   661]
 [13960   961]]
分类报告:
              precision    recall  f1-score   support

           0       0.53      0.96      0.69     16559
           1       0.59      0.06      0.12     14921

    accuracy                           0.54     31480
   macro avg       0.56      0.51      0.40     31480
weighted avg       0.56      0.54      0.42     31480

***********10日*************
模型准确率: 0.5355781448538754
混淆矩阵:
[[16218   344]
 [14276   642]]
分类报告:
              precision    recall  f1-score   support

   

1、所有时间段的报告显示负类样本远多于正类样本，导致模型偏向于预测负类。
2、20日最高价虽然准确率很高，但主要是因为模型几乎所有样本都预测为正类。负类样本几乎全被错判，召回率为0。

探究当日股价涨停和大盘对后续股价的预测准确率

In [24]:
df_best = df[df['Pct_Chg'] > 9.8]
print("***********3日*************")
df_best['Target_3d'] = (df_best['Pct_Chg_3d']-df_best['Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df_best, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_3d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("***********5日*************")
df_best['Target_5d'] = (df_best['Pct_Chg_5d']-df_best['Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df_best, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_5d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("***********10日*************")
df_best['Target_10d'] = (df_best['Pct_Chg_10d']-df_best['Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df_best, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_10d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("**********20日*************")
df_best['Target_20d'] = (df_best['Pct_Chg_20d']-df_best['Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df_best, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_20d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("**********20内的最高价日*************")
columns_to_check = ['Pct_Chg_3d', 'Pct_Chg_5d', 'Pct_Chg_10d', 'Pct_Chg_20d']
df_best['Max_Pct_Chg'] = df_best[columns_to_check].max(axis=1)
df_best['Target_max'] = (df_best['Max_Pct_Chg']-df_best['Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df_best, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_max')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])

***********3日*************
模型准确率: 0.674565560821485
混淆矩阵:
[[ 30 186]
 [ 20 397]]
分类报告:
              precision    recall  f1-score   support

           0       0.60      0.14      0.23       216
           1       0.68      0.95      0.79       417

    accuracy                           0.67       633
   macro avg       0.64      0.55      0.51       633
weighted avg       0.65      0.67      0.60       633

***********5日*************
模型准确率: 0.6808846761453397
混淆矩阵:
[[ 34 183]
 [ 19 397]]
分类报告:
              precision    recall  f1-score   support

           0       0.64      0.16      0.25       217
           1       0.68      0.95      0.80       416

    accuracy                           0.68       633
   macro avg       0.66      0.56      0.52       633
weighted avg       0.67      0.68      0.61       633

***********10日*************
模型准确率: 0.6761453396524486
混淆矩阵:
[[ 32 183]
 [ 22 396]]
分类报告:
              precision    recall  f1-score   support

           0       0.59    

D:\anaconda3\envs\strategy\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda3\envs\strategy\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\anaconda3\envs\strategy\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

类别不平衡: 每个时间段的结果显示负类样本的识别能力差，导致模型整体性能受限。负类的召回率普遍很低，而正类的召回率相对较高。

模型性能: 尽管在某些时间段准确率较高，但主要是因为模型偏向于预测正类样本，导致对负类的识别能力不足，可能会导致误导性的高准确率。

探究当日股价略涨和大盘对后续股价的预测准确率

In [25]:
df_lost = df[df['Pct_Chg'] < 9.8]
print("***********3日*************")
df_lost['Target_3d'] = (df_lost['Pct_Chg_3d']-df_lost['Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df_lost, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_3d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("***********5日*************")
df_lost['Target_5d'] = (df_lost['Pct_Chg_5d']-df_lost['Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df_lost, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_5d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("***********10日*************")
df_lost['Target_10d'] = (df_lost['Pct_Chg_10d']-df_lost['Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df_lost, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_10d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("**********20日*************")
df_lost['Target_20d'] = (df_lost['Pct_Chg_20d']-df_lost['Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df_lost, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_20d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("**********20内的最高价日*************")
columns_to_check = ['Pct_Chg_3d', 'Pct_Chg_5d', 'Pct_Chg_10d', 'Pct_Chg_20d']
df_lost['Max_Pct_Chg'] = df_lost[columns_to_check].max(axis=1)
df_lost['Target_max'] = (df_lost['Max_Pct_Chg']-df_lost['Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df_lost, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_max')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])

***********3日*************
模型准确率: 0.534995299380815
混淆矩阵:
[[16495     5]
 [14339     8]]
分类报告:
              precision    recall  f1-score   support

           0       0.53      1.00      0.70     16500
           1       0.62      0.00      0.00     14347

    accuracy                           0.53     30847
   macro avg       0.58      0.50      0.35     30847
weighted avg       0.57      0.53      0.37     30847

***********5日*************


D:\anaconda3\envs\strategy\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda3\envs\strategy\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\anaconda3\envs\strategy\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-def

模型准确率: 0.5327260349466723
混淆矩阵:
[[16433     0]
 [14414     0]]
分类报告:
              precision    recall  f1-score   support

           0       0.53      1.00      0.70     16433
           1       0.00      0.00      0.00     14414

    accuracy                           0.53     30847
   macro avg       0.27      0.50      0.35     30847
weighted avg       0.28      0.53      0.37     30847

***********10日*************
模型准确率: 0.5316886569196356
混淆矩阵:
[[16401     0]
 [14446     0]]
分类报告:
              precision    recall  f1-score   support

           0       0.53      1.00      0.69     16401
           1       0.00      0.00      0.00     14446

    accuracy                           0.53     30847
   macro avg       0.27      0.50      0.35     30847
weighted avg       0.28      0.53      0.37     30847

**********20日*************
模型准确率: 0.545563588031251
混淆矩阵:
[[16827     5]
 [14013     2]]
分类报告:
              precision    recall  f1-score   support

           0       0.55      1

D:\anaconda3\envs\strategy\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda3\envs\strategy\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda3\envs\strategy\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

所有时间段的结果显示，模型严重偏向于预测负类，导致正类的识别能力极差。负类的召回率为100%并不代表模型性能好，因为正类被完全忽略。